## Agents

In [2]:
## Wikipedia Wrapper
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun


In [12]:
# Wiki
api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Data Science\LangChain\GenAi_BotGemma\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
loader=WebBaseLoader("https://python.langchain.com/docs/integrations/tools/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"   # ✅ Gemini embedding model
))
retrevier=vectordb.as_retriever()
retrevier

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019A4D7354F0>, search_kwargs={})

In [8]:
## make retriever as create retrevier tool to ask qn
from langchain.tools.retriever import create_retriever_tool
retrever_tool1=create_retriever_tool(retrevier,"langchain_tool","Search for information about tools in langchain")

In [9]:
retrever_tool1.name

'langchain_tool'

In [10]:
#3 Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [26]:
import os
from langchain.tools import Tool
from langchain_exa import ExaSearchResults
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Initialize Exa tool with the API key
exa_tool = ExaSearchResults(exa_api_key=os.getenv("EXA_API_KEY"))

# Wrap in LangChain Tool
search_papers_tool = Tool(
    name="exa_research_papers",
    description="Search for academic and research papers on any topic.",
    func=lambda query: exa_tool._run(
        query=query,
        num_results=5,
        content_types=["research_paper"],   # 👈 ensure papers only
        text_contents_options=True,
        highlights=True,
    )
)


In [43]:
# tools=[exa_tool,wiki,arxiv,retrever_tool1]
tools=[exa_tool,wiki,retrever_tool1]

In [44]:
tools

[ExaSearchResults(client=<exa_py.api.Exa object at 0x0000019A525EBD70>, exa_api_key=SecretStr('**********')),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data Science\\LangChain\\GenAi_BotGemma\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='langchain_tool', description='Search for information about tools in langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000019A3537A700>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019A4D7354F0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), c

In [45]:
# using gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# ✅ Correct env variable
api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=api_key,   
    temperature=0.7
)

In [46]:
## Prompt
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [47]:
## Agents
from langchain.agents import create_openai_tools_agent
agents=create_openai_tools_agent(llm,tools,prompt)

In [48]:
## Agent Executer to run the agent 
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agents,tools=tools,verbose=True)

In [49]:
agent_executor.invoke({"input":"give the paper written by students or faculties of Sri Krishna Arts and Science College on AI"})



> Entering new AgentExecutor chain...

Invoking: `exa_search_results_json` with `{'query': 'AI research papers by students or faculty of Sri Krishna Arts and Science College'}`


Title: HOD's Desk - Sri Krishna Arts and Science College
URL: https://skasc.ac.in/discipline/computing-science/hods-desk/
ID: https://skasc.ac.in/discipline/computing-science/hods-desk/
Score: None
Published Date: 2024-10-14T06:53:16.000Z
Author: 
Image: None
Favicon: None
Extras: None
Subpages: None
Text: ![page-loader](data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20width='98vw'%20height='98vh'%20fill='none'%3E%3Crect%20width='98vw'%20height='98vh'%20/%3E%3C/svg%3E)

[Back to Disciplines](https://skasc.ac.in/academics/disciplines/)

# Computing Science

- [Overview](https://skasc.ac.in/discipline/computing-science/)
- [Department Highlights](https://skasc.ac.in/discipline/computing-science/department-highlights/)
- [Dean’s Desk](https://skasc.ac.in/discipline/computing-science/deans-desk/)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 125000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 7
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 125000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 4
}
]